In [1]:
#SETUP
import os
from openai import OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
# To finetune finetune_truth.jsonl on davinci-002 (the successor of curie, which is now deprecated)
client.files.create(
  file=open("TruthfulQA/data/finetune_truth.jsonl", "rb"),
  purpose="fine-tune"
)

client.fine_tuning.jobs.create(
  training_file=truth_file_id, 
  model="davinci-002",
  suffix="truthful",
  hyperparameters={
    "n_epochs":5,
    "batch_size":21,
    "learning_rate_multiplier":0.1
  }
)

In [ ]:
# Do the same for finetune_info to obtain GPT-info
client.files.create(
  file=open("TruthfulQA/data/finetune_info.jsonl", "rb"),
  purpose="fine-tune"
)

client.fine_tuning.jobs.create(
  training_file=truth_file_id, 
  model="davinci-002",
  suffix="informative",
  hyperparameters={
    "n_epochs":5,
    "batch_size":21,
    "learning_rate_multiplier":0.1
  }
)